In [5]:
import pandas as pd
from google.colab import drive
import glob

In [3]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd /content/drive/Shareddrives/Data606_Energy/data/merge_ready

/content/drive/Shareddrives/Data606_Energy/data/merge_ready


In [13]:
merge_files = glob.glob('*.csv')

In [75]:
first_year = 2008
last_year = 2021
for i, file in enumerate(merge_files):
  if 'merged' not in file:
    df = pd.read_csv(file)
    #first, make sure year column is named consistently
    try:
      df['Year']
    except:
      df['Year'] = df['year']
      del df['year']
    # now make sure state column is named consistently
    try:
      df['State']
    except:
      try:
        df['State'] = df['state']
        del df['state']
      except:
        ...# this is in the case that there isn't state in the dataframe - futures data doesn't have state for example
    # now subset the dataframe to only have data for the time window between first and last year
    df = df[(df['Year']>=first_year)&(df['Year']<=last_year)]
    # I'm doing a groupby so that indices will match up nicely for the joins
    # also because the electricity data is still at a monthly level, so averaging it seems appropriate to get it to annual
    if 'State' in df.columns:
      df = df.groupby(['Year','State']).mean()
    else:
      df = df.groupby(['Year']).mean()
    # now try to merge the data. If merge data doesn't exist yet then create it
    if i ==0:
      merged_data = df.copy()
    else:
      merged_data = pd.merge(merged_data,df,left_index=True, right_index=True)#on=['Year','State'])#,how='outer')

In [76]:
merged_data.head()

kWh_percapita_electric_accounts  ...  Months Above Average Price Volume HO=F
Year State                                   ...                                        
2010 AK                        2.250749e+04  ...                                       6
     AL                        9.997777e+04  ...                                       6
     AR                        1.408243e+04  ...                                       6
     AZ                        3.617310e+04  ...                                       6
     CA                        1.069094e+06  ...                                       6

[5 rows x 15 columns]

In [78]:
merged_data.to_csv('merged_data.csv')